# Advanced Applications of MOPSO & MONLTA

This notebook extends the core optimization framework to five real-world glass melter scenarios from the second research notebook:

| # | Application | Decision Variables | Objectives |
|---|---|---|---|
| 1 | Neural ODE Hyper-parameter Tuning | learning rate, hidden size, epochs | Val. MSE, Training Time, Complexity |
| 2 | Luenberger Observer Gain Selection | L gains (7-state) | Estimation Error, Robustness, Settling |
| 3 | MPC Weight Tuning | Q, R, N weights | Tracking Error, Control Effort, Constraint Violation |
| 4 | Fractional-Order PID (FOPID) | Kp, Ki, Kd, λ, μ | IAE, Overshoot, Settling |
| 5 | Multi-Zone Temperature PID | 3×(Kp,Ki,Kd) = 9 vars | Zone IAE_1, IAE_2, IAE_3 |

Each application reuses the `mopso_monlta` package with custom objective functions.

In [ ]:
import numpy as np
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from mopso_monlta.optimizers import MOPSO, MONLTA
from mopso_monlta.evaluation import get_pareto_front, select_best_compromise
from mopso_monlta.visualization import setup_publication_style, COLORS, plot_pareto_2d

setup_publication_style()
np.random.seed(42)
print('Imports OK')

---
## Application 1: Neural ODE Hyper-parameter Tuning

Optimize learning rate ($\eta$), hidden-layer size ($n_h$), and number of epochs.

In [ ]:
def neural_ode_objectives(x):
    """Surrogate objectives for Neural ODE hyper-parameter tuning."""
    lr, hidden, epochs = x
    hidden = int(round(hidden))
    epochs = int(round(epochs))
    
    # Surrogate: validation MSE ~ U-shaped in lr, decreasing in capacity
    val_mse = 0.01 / (lr + 0.001) + 0.001 * lr + 0.5 / hidden + np.random.normal(0, 0.002)
    # Training time ~ linear in hidden * epochs
    train_time = hidden * epochs * 0.001 + np.random.normal(0, 0.1)
    # Complexity ~ hidden^2
    complexity = hidden ** 2 / 1000.0
    
    return [max(val_mse, 0), max(train_time, 0), complexity]

bounds_node = [(1e-4, 0.1), (16, 256), (50, 500)]

mopso_node = MOPSO(neural_ode_objectives, bounds_node, n_particles=30, n_objectives=3)
mopso_node.optimize(n_iterations=50)

monlta_node = MONLTA(neural_ode_objectives, bounds_node, n_objectives=3)
monlta_node.optimize(n_episodes=4, steps_per_episode=50)

plot_pareto_2d(
    {'MOPSO': mopso_node.archive_objectives, 'MONLTA': monlta_node.archive_objectives},
    obj_names=('Val MSE', 'Train Time (s)', 'Complexity'),
)

best_g, best_o = select_best_compromise(mopso_node.archive_positions, mopso_node.archive_objectives)
print(f'Best Neural ODE config: lr={best_g[0]:.5f}, hidden={int(best_g[1])}, epochs={int(best_g[2])}')
print(f'Objectives: MSE={best_o[0]:.4f}, Time={best_o[1]:.2f}s, Complexity={best_o[2]:.3f}')

---
## Application 2: Luenberger Observer Gain Selection

Optimize observer gains $L_1, \dots, L_4$ for the 7-state glass melter model.

In [ ]:
from mopso_monlta.config import GlassMelterParams7State

def observer_objectives(x):
    """Surrogate for Luenberger observer gain tuning."""
    L = x  # observer gains
    
    # Estimation error ~ decreases with gain magnitude, but noise amplification increases
    gain_norm = np.linalg.norm(L)
    est_error = 1.0 / (gain_norm + 0.1) + np.random.normal(0, 0.005)
    # Robustness ~ inversely related to max gain
    robustness = np.max(np.abs(L)) / 10.0 + np.random.normal(0, 0.01)
    # Settling time ~ related to smallest gain
    settling = 5.0 / (np.min(np.abs(L)) + 0.01) + np.random.normal(0, 0.1)
    
    return [max(est_error, 0), max(robustness, 0), max(settling, 0)]

bounds_obs = [(0.1, 10.0)] * 4

mopso_obs = MOPSO(observer_objectives, bounds_obs, n_particles=25, n_objectives=3)
mopso_obs.optimize(n_iterations=50)

monlta_obs = MONLTA(observer_objectives, bounds_obs, n_objectives=3)
monlta_obs.optimize(n_episodes=4, steps_per_episode=50)

plot_pareto_2d(
    {'MOPSO': mopso_obs.archive_objectives, 'MONLTA': monlta_obs.archive_objectives},
    obj_names=('Est. Error', 'Noise Sensitivity', 'Settling Time'),
)

best_g, best_o = select_best_compromise(mopso_obs.archive_positions, mopso_obs.archive_objectives)
print(f'Best observer gains: L = {np.round(best_g, 4)}')

---
## Application 3: MPC Weight Tuning

Optimize $Q$ (tracking weight), $R$ (control effort weight), and $N$ (prediction horizon).

In [ ]:
def mpc_objectives(x):
    """Surrogate for MPC weight optimization."""
    Q, R, N = x
    N = int(round(N))
    
    tracking = 0.5 / (Q + 0.01) + 0.01 * R + np.random.normal(0, 0.005)
    effort = Q / (R + 0.01) * 0.1 + np.random.normal(0, 0.01)
    constraint_viol = max(0, 0.2 - 0.005 * N + 0.01 * Q / R) + np.random.normal(0, 0.005)
    
    return [max(tracking, 0), max(effort, 0), max(constraint_viol, 0)]

bounds_mpc = [(0.1, 100.0), (0.01, 50.0), (5, 50)]

mopso_mpc = MOPSO(mpc_objectives, bounds_mpc, n_particles=25, n_objectives=3)
mopso_mpc.optimize(n_iterations=50)

monlta_mpc = MONLTA(mpc_objectives, bounds_mpc, n_objectives=3)
monlta_mpc.optimize(n_episodes=4, steps_per_episode=50)

plot_pareto_2d(
    {'MOPSO': mopso_mpc.archive_objectives, 'MONLTA': monlta_mpc.archive_objectives},
    obj_names=('Tracking Error', 'Control Effort', 'Constraint Violation'),
)

best_g, best_o = select_best_compromise(mopso_mpc.archive_positions, mopso_mpc.archive_objectives)
print(f'Best MPC weights: Q={best_g[0]:.3f}, R={best_g[1]:.3f}, N={int(best_g[2])}')

---
## Application 4: Fractional-Order PID (FOPID)

Extend standard PID with fractional integrator order $\lambda$ and differentiator order $\mu$:

$$C(s) = K_p + \frac{K_i}{s^\lambda} + K_d s^\mu$$

In [ ]:
from mopso_monlta.evaluation.objectives import simulate_closed_loop

def fopid_objectives(x):
    """Objectives for FOPID tuning (5 decision variables)."""
    Kp, Ki, Kd, lam, mu = x
    
    # Approximate fractional PID via modified gains
    Ki_eff = Ki * (1.0 + 0.2 * (lam - 1.0))  # fractional correction
    Kd_eff = Kd * (1.0 + 0.2 * (mu - 1.0))
    
    try:
        metrics = simulate_closed_loop([Kp, max(Ki_eff, 0.001), max(Kd_eff, 0)])
        return [metrics['IAE'], metrics['overshoot'], metrics['settling_time']]
    except Exception:
        return [1e6, 1e6, 1e6]

bounds_fopid = [(0.1, 20), (0.001, 5), (0, 10), (0.5, 1.5), (0.5, 1.5)]

mopso_fo = MOPSO(fopid_objectives, bounds_fopid, n_particles=30, n_objectives=3)
mopso_fo.optimize(n_iterations=50)

monlta_fo = MONLTA(fopid_objectives, bounds_fopid, n_objectives=3)
monlta_fo.optimize(n_episodes=4, steps_per_episode=50)

plot_pareto_2d(
    {'MOPSO': mopso_fo.archive_objectives, 'MONLTA': monlta_fo.archive_objectives},
    obj_names=('IAE', 'Overshoot (%)', 'Settling Time (h)'),
)

best_g, best_o = select_best_compromise(mopso_fo.archive_positions, mopso_fo.archive_objectives)
print(f'Best FOPID: Kp={best_g[0]:.3f}, Ki={best_g[1]:.4f}, Kd={best_g[2]:.3f}, λ={best_g[3]:.3f}, μ={best_g[4]:.3f}')

---
## Application 5: Multi-Zone Temperature PID

Tune 3 independent PID controllers (9 decision variables) for a multi-zone glass melter.

In [ ]:
def multizone_objectives(x):
    """Objectives for 3-zone PID tuning (9 decision variables)."""
    zone_iae = []
    for zone in range(3):
        Kp, Ki, Kd = x[zone*3:(zone+1)*3]
        try:
            metrics = simulate_closed_loop([Kp, Ki, Kd])
            zone_iae.append(metrics['IAE'] * (1.0 + 0.1 * zone))  # zone coupling
        except Exception:
            zone_iae.append(1e6)
    return zone_iae

bounds_mz = [(0.1, 20), (0.001, 5), (0, 10)] * 3  # 9 variables

mopso_mz = MOPSO(multizone_objectives, bounds_mz, n_particles=40, n_objectives=3, archive_size=150)
mopso_mz.optimize(n_iterations=60)

monlta_mz = MONLTA(multizone_objectives, bounds_mz, n_objectives=3)
monlta_mz.optimize(n_episodes=5, steps_per_episode=60)

plot_pareto_2d(
    {'MOPSO': mopso_mz.archive_objectives, 'MONLTA': monlta_mz.archive_objectives},
    obj_names=('Zone 1 IAE', 'Zone 2 IAE', 'Zone 3 IAE'),
)

best_g, best_o = select_best_compromise(mopso_mz.archive_positions, mopso_mz.archive_objectives)
for z in range(3):
    Kp, Ki, Kd = best_g[z*3:(z+1)*3]
    print(f'Zone {z+1}: Kp={Kp:.3f}, Ki={Ki:.4f}, Kd={Kd:.3f} | IAE={best_o[z]:.4f}')

---
## Summary

All five applications demonstrate that `mopso_monlta` is a general-purpose multi-objective optimization toolkit.
The only change between applications is the **objective function** and **bounds** — the optimizers remain the same.